In [1]:
## Bring in needed mods
import pandas as pd, numpy as np, glob, sys, os, seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.insert(0, '/home/croth/QTL-functions/SCRIPTS/')
import qtlfunctions as qtlfun

In [2]:
## Set variables
## Set chromosomes
chromns = np.arange(1,15)

## path to info dataframes
infopath = '../GENOTYPE/INFO/*.%s.B3502.progeny_info_cols.csv.gz'

## path to genotype dataframes
genopath = '../GENOTYPE/GT/*.%s.B3502.progeny_genotypes.csv.gz'

## path to depth dataframes
deptpath = '../GENOTYPE/DP/*.%s.B3502.progeny_depths.csv.gz'

## path to allelic dataframes
allepath = '../GENOTYPE/AF/*.%s.B3502.progeny_allele_ratios.csv.gz'

In [19]:
## Load in depth dataframe
dps = qtlfun.loadvariants(infopath,deptpath,chromns)

## View tail
dps.tail().T.tail().T

,A30,A31,A33,A32,A34
22654,91,78,73,130,78
22656,94,62,77,125,80
22671,28,22,14,26,28
22682,4,1,0,3,3
22683,0,0,0,0,0


In [20]:
## Load in allelic ratio dataframe
ads = qtlfun.loadvariants(infopath,allepath,chromns)

## View tail
ads.tail().T.tail().T

,A30,A31,A33,A32,A34
22654,1,1,1,1,1
22656,1,1,1,1,1
22671,1,1,1,1,1
22682,1,1,NaN,1,1
22683,NaN,NaN,NaN,NaN,NaN


In [16]:
## Gather sample names and print length
samples = ads.columns[ads.columns.tolist().index('MAF')+1:].tolist()
print(len(samples))

92


In [17]:
## Gather progeny
progeny = sorted([s for s in samples if len(s)<=4])

## Gather ancestors
ancestors = sorted([s for s in samples if len(s)>4])

## Check work
assert len(progeny) + len(ancestors) == len(samples)

## Gather B3502 stocks
b3502 = [s for s in ancestors if s.split("_")[0]=='B3502']

## parents and progenitor strains
progenitors = [s for s in ancestors if s not in b3502]

In [7]:
## For each sample, plot the depth and allelic depth ratio
for sample in samples:

    ## Call figure
    fig,ax = plt.subplots(2,1,figsize=(12,5),sharex=True)
    fig.set_facecolor('w')

    ## Set axis and title
    plt.sca(ax[0])
    plt.title(sample)

    ## plot log2 depth
    qtlfun.manhattan(dps,pval=sample,chrom='Seqid',xlabel='',
                 ylabel='log$_2$ (Depth)',ylims=(0,12),ax=ax[0]);

    ## plot allelic depth ratio
    qtlfun.manhattan(ads,pval=sample,chrom='Seqid',chromlabels=chromns,
                 ylabel='Allelic Depth',ylims=(0,1),ax=ax[1]);
    
    ## Save figure and close figure
    plt.savefig('../FIGURES/DEPTHPLOTS/%s.png'%sample,
                dpi=100,bbox_inches='tight')
    plt.close()